In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
#Thêm một số thư viện cần thiết
import pandas as pd

import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from math import sqrt
from numpy.linalg import norm
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, rank, monotonically_increasing_id 
from operator import add
from pyspark.sql import Row

<h1>1) Read CSV from Google Cloud Storage

In [3]:
events_df=spark.read.csv('gs://bigdataprojectbk/event.csv', header=True, inferSchema=True)

In [4]:
events_df.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: int, id_odsp: string, id_event: string, sort_order: int, time: int, text: string, event_type: int, event_type2: double, side: int, event_team: string, opponent: string, player: string, player2: string, player_in: string, player_out: string, shot_place: double, shot_outcome: double, is_goal: int, location: double, bodypart: double, assist_method: int, situation: double, fast_break: int, game_id: int]>

In [5]:
game_df=spark.read.csv('gs://bigdataprojectbk/ginf.csv', header=True, inferSchema=True)

In [6]:
game_df.printSchema

<bound method DataFrame.printSchema of DataFrame[id_odsp: string, link_odsp: string, adv_stats: boolean, date: timestamp, league: string, season: int, country: string, ht: string, at: string, fthg: int, ftag: int, odd_h: double, odd_d: double, odd_a: double, odd_over: string, odd_under: string, odd_bts: string, odd_bts_n: string]>

<h1>2) Data Transformation

Map the nummeric code in the event dataframe to description string

In [7]:
#Define a generic dictionary/map lookup function
def mapKeyToVal(mapping):
    def mapKeyToVal_(col):
        return mapping.get(col)
    return udf(mapKeyToVal_, StringType())

In [8]:
#Create dictionary
evtTypeMap = {0:'Announcement', 1:'Attempt', 2:'Corner', 3:'Foul', 4:'Yellow card', 5:'Second yellow card', 6:'Red card', 7:'Substitution', 8:'Free kick won', 9:'Offside', 10:'Hand ball', 11:'Penalty conceded', 99:'NA'}

evtTyp2Map = {12:'Key Pass', 13:'Failed through ball', 14:'Sending off', 15:'Own goal', 0:'NA'}

sideMap = {1:'Home', 2:'Away'}

shotPlaceMap = {1:'Bit too high', 2:'Blocked', 3:'Bottom left corner', 4:'Bottom right corner', 5:'Centre of the goal', 6:'High and wide', 7:'Hits the bar', 8:'Misses to the left', 9:'Misses to the right', 10:'Too high', 11:'Top centre of the goal', 12:'Top left corner', 13:'Top right corner', 0:'NA'}

shotOutcomeMap = {1:'On target', 2:'Off target', 3:'Blocked', 4:'Hit the bar', 0:'NA'}

locationMap = {1:'Attacking half', 2:'Defensive half', 3:'Centre of the box', 4:'Left wing', 5:'Right wing', 6:'Difficult angle and long range', 7:'Difficult angle on the left', 8:'Difficult angle on the right', 9:'Left side of the box', 10:'Left side of the six yard box', 11:'Right side of the box', 12:'Right side of the six yard box', 13:'Very close range', 14:'Penalty spot', 15:'Outside the box', 16:'Long range', 17:'More than 35 yards', 18:'More than 40 yards', 19:'Not recorded', 0:'NA'}

bodyPartMap = {1:'Right foot', 2:'Left foot', 3:'Head', 0:'NA'}

assistMethodMap = {0:'None', 1:'Pass', 2:'Cross', 3:'Headed pass', 4:'Through ball', 0:'NA'}

situationMap = {1:'Open play', 2:'Set piece', 3:'Corner', 4:'Free kick', 0:'NA'}

countryCodeMap = {'germany':'DEU', 'france':'FRA', 'england':'GBR', 'spain':'ESP', 'italy':'ITA'}
countryIDMap = {'DEU':'1', 'FRA':'2', 'GBR':'3', 'ESP':'4', 'ITA':'5'}

In [9]:
#Map to country code
game_df = game_df.withColumn("country_code", mapKeyToVal(countryCodeMap)("country"))
game_df = game_df.withColumn("country_ID", mapKeyToVal(countryIDMap)("country_code"))


In [10]:
#Transform game events data with dictionary info
events_df = (
             events_df.
             withColumn("event_type_str", mapKeyToVal(evtTypeMap)("event_type")).
             withColumn("event_type2_str", mapKeyToVal(evtTyp2Map)("event_type2")).
             withColumn("side_str", mapKeyToVal(sideMap)("side")).
             withColumn("shot_place_str", mapKeyToVal(shotPlaceMap)("shot_place")).
             withColumn("shot_outcome_str", mapKeyToVal(shotOutcomeMap)("shot_outcome")).
             withColumn("location_str", mapKeyToVal(locationMap)("location")).
             withColumn("bodypart_str", mapKeyToVal(bodyPartMap)("bodypart")).
             withColumn("assist_method_str", mapKeyToVal(assistMethodMap)("assist_method")).
             withColumn("situation_str", mapKeyToVal(situationMap)("situation"))
           )

joinedDf = (
  events_df.join(game_df, events_df.id_odsp == game_df.id_odsp, 'inner').
  select(events_df.game_id, events_df.id_event, events_df.sort_order, events_df.time, events_df.event_type, events_df.event_type_str, events_df.event_type2, events_df.event_type2_str, events_df.side, events_df.side_str, events_df.event_team, events_df.opponent, events_df.player, events_df.player2, events_df.player_in, events_df.player_out, events_df.shot_place, events_df.shot_place_str, events_df.shot_outcome, events_df.shot_outcome_str, events_df.is_goal, events_df.location, events_df.location_str, events_df.bodypart, events_df.bodypart_str, events_df.assist_method, events_df.assist_method_str, events_df.situation, events_df.situation_str, game_df.country_code,game_df.country_ID,game_df.date)
)

In [11]:
joinedDf.printSchema

<bound method DataFrame.printSchema of DataFrame[game_id: int, id_event: string, sort_order: int, time: int, event_type: int, event_type_str: string, event_type2: double, event_type2_str: string, side: int, side_str: string, event_team: string, opponent: string, player: string, player2: string, player_in: string, player_out: string, shot_place: double, shot_place_str: string, shot_outcome: double, shot_outcome_str: string, is_goal: int, location: double, location_str: string, bodypart: double, bodypart_str: string, assist_method: int, assist_method_str: string, situation: double, situation_str: string, country_code: string, country_ID: string, date: timestamp]>

Create time bin for the event

In [12]:
#Create time bin
from pyspark.ml.feature import QuantileDiscretizer

joinedDf = QuantileDiscretizer(numBuckets=10, inputCol="time", outputCol="time_bin").fit(joinedDf).transform(joinedDf)


In [13]:
#cast event back to nummeric
joinedDf = joinedDf.withColumn("event_type2",col("event_type2").cast(IntegerType())) 
joinedDf = joinedDf.withColumn("shot_place",col("shot_place").cast(IntegerType()))
joinedDf = joinedDf.withColumn("shot_outcome",col("shot_outcome").cast(IntegerType()))
joinedDf = joinedDf.withColumn("location",col("location").cast(IntegerType()))
joinedDf = joinedDf.withColumn("bodypart",col("bodypart").cast(IntegerType()))
joinedDf = joinedDf.withColumn("situation",col("situation").cast(IntegerType()))
joinedDf = joinedDf.withColumn("date",col("date").cast(DateType()))



In [14]:
joinedDf.printSchema

<bound method DataFrame.printSchema of DataFrame[game_id: int, id_event: string, sort_order: int, time: int, event_type: int, event_type_str: string, event_type2: int, event_type2_str: string, side: int, side_str: string, event_team: string, opponent: string, player: string, player2: string, player_in: string, player_out: string, shot_place: int, shot_place_str: string, shot_outcome: int, shot_outcome_str: string, is_goal: int, location: int, location_str: string, bodypart: int, bodypart_str: string, assist_method: int, assist_method_str: string, situation: int, situation_str: string, country_code: string, country_ID: string, date: date, time_bin: double]>

In [15]:
joinedDf.limit(3).toPandas()

,game_id,id_event,sort_order,time,event_type,event_type_str,event_type2,event_type2_str,side,side_str,...,bodypart,bodypart_str,assist_method,assist_method_str,situation,situation_str,country_code,country_ID,date,time_bin
0,4483,UFot0hit1,1,2,1,Attempt,12,Key Pass,2,Away,...,2,Left foot,1,Pass,1,Open play,DEU,1,2011-08-05,0.0
1,4483,UFot0hit2,2,4,2,Corner,0,NA,1,Home,...,0,NA,0,NA,0,NA,DEU,1,2011-08-05,0.0
2,4483,UFot0hit3,3,4,2,Corner,0,NA,1,Home,...,0,NA,0,NA,0,NA,DEU,1,2011-08-05,0.0


<h1>3. Exploratory Data Analysis (EDA)

Create a SQL view from Spark Dataframe

In [ ]:
joinedDf

In [16]:
joinedDf.createOrReplaceTempView("EuropeFootball")

Các trận đấu có nhiều bàn nhất

In [70]:
spark.sql("SELECT game_id,SUM(is_goal) AS GOAL FROM EuropeFootball GROUP BY game_id ORDER BY GOAL DESC LIMIT 5").show()

+-------+----+
|game_id|GOAL|
+-------+----+
|   4657|  12|
|   6862|  11|
|   7393|  10|
|   7353|  10|
|   2053|  10|
+-------+----+



In [73]:
spark.sql("SELECT event_team, opponent,date FROM EuropeFootball WHERE game_id=4657 LIMIT 1").show()

+-----------+--------------+----------+
| event_team|      opponent|      date|
+-----------+--------------+----------+
|Real Madrid|Rayo Vallecano|2015-12-20|
+-----------+--------------+----------+



Trung bình số bàn mỗi trận

In [69]:
spark.sql("SELECT (SUM(is_goal)/COUNT(DISTINCT game_id)) AS AVERAGE_GOAL_PER_GAME FROM EuropeFootball").show()

+---------------------+
|AVERAGE_GOAL_PER_GAME|
+---------------------+
|   2.6940709720079346|
+---------------------+



Bàn thắng ở các phút

In [62]:
spark.sql("SELECT time_bin,SUM(is_goal) AS GOAL FROM EuropeFootball GROUP BY time_bin ORDER BY GOAL DESC LIMIT 10").show()

+--------+----+
|time_bin|GOAL|
+--------+----+
|     9.0|2688|
|     2.0|2636|
|     4.0|2572|
|     8.0|2541|
|     5.0|2530|
|     6.0|2507|
|     1.0|2331|
|     3.0|2245|
|     7.0|2235|
|     0.0|2161|
+--------+----+



Body part score most goal

In [61]:
spark.sql("SELECT bodypart_str,SUM(is_goal) AS GOAL FROM EuropeFootball GROUP BY bodypart_str ORDER BY GOAL DESC LIMIT 10").show()

+------------+-----+
|bodypart_str| GOAL|
+------------+-----+
|  Right foot|13451|
|   Left foot| 6758|
|        Head| 4236|
|          NA|    1|
+------------+-----+



Top goal scorer across europe

In [17]:
spark.sql("SELECT player,SUM(is_goal) AS GOAL FROM EuropeFootball GROUP BY player ORDER BY GOAL DESC LIMIT 10").show()

+--------------------+----+
|              player|GOAL|
+--------------------+----+
|        lionel messi| 205|
|   cristiano ronaldo| 198|
|  zlatan ibrahimovic| 153|
|  robert lewandowski| 124|
|      edinson cavani| 121|
|     gonzalo higuain| 118|
|pierreemerick aub...| 100|
|         luis suarez|  96|
|         diego costa|  93|
| alexandre lacazette|  88|
+--------------------+----+



Top assistant across europe


In [18]:
spark.sql("SELECT player2,SUM(is_goal) AS ASSIST FROM EuropeFootball WHERE player2<>'0' GROUP BY player2 ORDER BY ASSIST DESC LIMIT 10").show()

+-----------------+------+
|          player2|ASSIST|
+-----------------+------+
|     lionel messi|    75|
|   angel di maria|    69|
|   gonzalo castro|    66|
|    dimitri payet|    60|
|    cesc fabregas|    60|
|       mesut ozil|    57|
|     marek hamsik|    56|
|cristiano ronaldo|    53|
|    franck ribery|    51|
|  kevin de bruyne|    51|
+-----------------+------+



Team scored the most goal from open play

In [19]:
spark.sql("SELECT event_team,SUM(is_goal) AS GOAL FROM EuropeFootball WHERE situation_str='Open play' GROUP BY event_team ORDER BY GOAL DESC LIMIT 10").show()

+-------------------+----+
|         event_team|GOAL|
+-------------------+----+
|          Barcelona| 472|
|        Real Madrid| 429|
|      Bayern Munich| 357|
|Paris Saint-Germain| 327|
|             Napoli| 319|
|  Borussia Dortmund| 297|
|           Juventus| 290|
|            AS Roma| 265|
|               Lyon| 246|
|    Atletico Madrid| 243|
+-------------------+----+



Team scored the most goal from set piece

In [20]:
spark.sql("SELECT event_team,SUM(is_goal) AS GOAL FROM EuropeFootball WHERE situation_str='Free kick' OR situation_str='Set piece'  GROUP BY event_team ORDER BY GOAL DESC LIMIT 10").show()

+-------------------+----+
|         event_team|GOAL|
+-------------------+----+
|        Real Madrid|  94|
|Paris Saint-Germain|  88|
|          Barcelona|  80|
|            AS Roma|  73|
|           AC Milan|  71|
|               Lyon|  70|
|           Juventus|  69|
|      Bayern Munich|  66|
|         Fiorentina|  64|
|              Lazio|  63|
+-------------------+----+



Team scored the most goal from corner

In [21]:
spark.sql("SELECT event_team,SUM(is_goal) AS GOAL FROM EuropeFootball WHERE situation_str='Corner' GROUP BY event_team ORDER BY GOAL DESC LIMIT 10").show()

+-----------------+----+
|       event_team|GOAL|
+-----------------+----+
|  Atletico Madrid|  51|
|      Real Madrid|  50|
|  Athletic Bilbao|  48|
|         Juventus|  42|
|       Fiorentina|  41|
|         AC Milan|  40|
|Borussia Dortmund|  40|
|         Valencia|  40|
|          Sevilla|  40|
|             Lyon|  39|
+-----------------+----+



Teams love to walk it in

In [22]:
spark.sql("SELECT event_team,SUM(is_goal) AS GOAL FROM EuropeFootball WHERE location_str='Very close range' GROUP BY event_team ORDER BY GOAL DESC LIMIT 10").show()

+-------------------+----+
|         event_team|GOAL|
+-------------------+----+
|        Real Madrid|  77|
|          Barcelona|  72|
|Paris Saint-Germain|  67|
|      Bayern Munich|  61|
|           Valencia|  58|
|    Atletico Madrid|  56|
|            Sevilla|  54|
|             Napoli|  53|
|     Internazionale|  53|
|              Lazio|  52|
+-------------------+----+



Penalty mechant

In [23]:
spark.sql("SELECT player,SUM(is_goal) AS GOAL FROM EuropeFootball WHERE location=14 GROUP BY player ORDER BY GOAL DESC LIMIT 10").show()

+-------------------+----+
|             player|GOAL|
+-------------------+----+
|  cristiano ronaldo|  43|
| zlatan ibrahimovic|  35|
|       lionel messi|  30|
|     edinson cavani|  20|
|             falcao|  20|
|    sejad salihovic|  19|
|alexandre lacazette|  18|
|   antonio candreva|  18|
|        eden hazard|  18|
|   domenico berardi|  16|
+-------------------+----+



Set piece mechant

In [24]:
spark.sql("SELECT player,SUM(is_goal) AS GOAL FROM EuropeFootball WHERE situation=4 GROUP BY player ORDER BY GOAL DESC LIMIT 10").show()

+------------------+----+
|            player|GOAL|
+------------------+----+
|      lionel messi|  14|
| cristiano ronaldo|  13|
|    miralem pjanic|  13|
|      andrea pirlo|  12|
|    francesco lodi|  10|
|  hakan calhanoglu|  10|
|             benat|   9|
|       daniel wass|   9|
|zlatan ibrahimovic|   8|
|       juan arango|   7|
+------------------+----+



Players with most own goal

In [25]:
spark.sql("SELECT player,SUM(is_goal) AS GOAL FROM EuropeFootball WHERE event_type2=15 GROUP BY player ORDER BY GOAL DESC LIMIT 10").show()

+-----------------+----+
|           player|GOAL|
+-----------------+----+
|   gareth mcauley|   4|
|    anthony weber|   4|
|   vangelis moras|   4|
|    martin skrtel|   4|
|       harry kane|   3|
|       john terry|   3|
|     aly cissokho|   3|
|alejandro arribas|   3|
|    davide astori|   3|
| shkodran mustafi|   3|
+-----------------+----+



In [26]:
spark.sql("SELECT player,SUM(is_goal) AS GOAL FROM EuropeFootball WHERE event_type2=15 GROUP BY player ORDER BY GOAL DESC LIMIT 10").show()

+---------------+----+
|         player|GOAL|
+---------------+----+
| gareth mcauley|   4|
|  martin skrtel|   4|
|  anthony weber|   4|
| vangelis moras|   4|
|paolo cannavaro|   3|
|aymen abdennour|   3|
|  davide astori|   3|
|     harry kane|   3|
|    diego mainz|   3|
|     john terry|   3|
+---------------+----+



<h1>4. Machine learning

XGoal Prediction

In [27]:
joinedDf.limit(3).toPandas()

,game_id,id_event,sort_order,time,event_type,event_type_str,event_type2,event_type2_str,side,side_str,...,bodypart,bodypart_str,assist_method,assist_method_str,situation,situation_str,country_code,country_ID,date,time_bin
0,4483,UFot0hit1,1,2,1,Attempt,12,Key Pass,2,Away,...,2,Left foot,1,Pass,1,Open play,DEU,1,2011-08-05,0.0
1,4483,UFot0hit2,2,4,2,Corner,0,NA,1,Home,...,0,NA,0,NA,0,NA,DEU,1,2011-08-05,0.0
2,4483,UFot0hit3,3,4,2,Corner,0,NA,1,Home,...,0,NA,0,NA,0,NA,DEU,1,2011-08-05,0.0


<h3>Train/test manual split

In [28]:
def cos(a,b):
    s = 0
    for i,j in zip(a,b):
        s = s + i*j
    return s/(norm(a)*norm(b))

In [29]:
def cos_map(df):
    new_rdd = df.rdd.map(lambda x: (cosine(x.feature,x.test_feature), x.is_goal, x.test_id))
    return new_rdd 

In [30]:
use_cols=joinedDf.select("country_code","shot_place","shot_outcome","location","bodypart","assist_method","situation","is_goal")


In [31]:
actions = use_cols.select("country_code","is_goal", f.array(use_cols.columns[1:]).alias("feature"))
actions.show(5)

+------------+-------+--------------------+
|country_code|is_goal|             feature|
+------------+-------+--------------------+
|         DEU|      0|[6, 2, 9, 2, 1, 1...|
|         DEU|      0|[0, 0, 0, 0, 0, 0...|
|         DEU|      0|[0, 0, 0, 0, 0, 0...|
|         DEU|      0|[0, 0, 0, 0, 0, 0...|
|         DEU|      0|[0, 0, 2, 0, 0, 0...|
+------------+-------+--------------------+
only showing top 5 rows



In [32]:
actions=actions.dropDuplicates(['feature'])

In [33]:
actions.createOrReplaceTempView("actions")

In [34]:
action_df=spark.sql("SELECT ROW_NUMBER() OVER(ORDER BY country_code) AS ActionID,country_code,is_goal,feature FROM actions")

In [35]:
action_df

DataFrame[ActionID: int, country_code: string, is_goal: int, feature: array<int>]

In [36]:
action_df.limit(4).toPandas()

,ActionID,country_code,is_goal,feature
0,1,DEU,0,"[1, 2, 11, 3, 1, 1, 0]"
1,2,DEU,1,"[3, 1, 12, 1, 1, 2, 1]"
2,3,DEU,0,"[9, 2, 16, 2, 0, 1, 0]"
3,4,DEU,0,"[10, 2, 13, 3, 2, 2, 0]"


In [37]:
train, test = action_df.randomSplit([0.9, 0.1])

In [38]:
train.take(2)

[Row(ActionID=1, country_code='DEU', is_goal=0, feature=[1, 2, 11, 3, 1, 1, 0]),
 Row(ActionID=2, country_code='DEU', is_goal=1, feature=[3, 1, 12, 3, 2, 2, 1])]

In [39]:
def get_game_train_vectors(df, country_code):
    return df.filter(df.country_code == country_code).select(col("feature"), col("is_goal"))

In [40]:
def get_game_test_vectors(df, country_code):
    return df.filter(df.country_code == country_code).select(col("feature").alias("test_feature"), col("ActionID").alias("test_id")).dropDuplicates()

In [41]:
train_vec = get_game_train_vectors(train, 'DEU')
train_vec.take(1)

[Row(feature=[3, 1, 6, 1, 1, 1, 0], is_goal=0)]

In [42]:
test_vec = get_game_test_vectors(test, 'DEU')
test_vec.take(1)

[Row(test_feature=[5, 1, 3, 1, 1, 2, 1], test_id=6)]

In [56]:
class KNN_goal_predict:
    #khởi tạo giá trị ban đầu
    def __init__(self, country_code, train_vec, test_vec, k):
        self.country_code = country_code
        self.train_vec = train_vec
        self.test_vec = test_vec
        self.k = k
        
    def knn_result(self):
        k = self.k
        tr = self.train_vec
        td = self.test_vec
        join_df = tr.crossJoin(td)
        #tính cosin giữa vector test và các vector train, thông qua rdd
        cos_map_df = join_df\
                    .rdd.map(lambda x: (float(cos(x.feature,x.test_feature)), x.is_goal, x.test_id))\
                    .toDF(["cos", "is_goal", "test_id"])
        #windowDept short lại cái cos score ở trên
        windowDept = Window.partitionBy("test_id").orderBy(col("cos").desc(), col("is_goal").desc())
    
        top_k_nearest_items_df = cos_map_df\
                                .withColumn("row",row_number().over(windowDept)).filter(col("row") <= k)
        #top_k_nearest: dựa vào list đã sort ở trên, tìm k điểm gần nhất
        xG_df = top_k_nearest_items_df.rdd.map(lambda x: (x.test_id, x.cos*x.is_goal))\
                                                    .reduceByKey(add)\
                                                    .map(lambda x: (x[0], (x[1]/k)))\
                                                    .toDF(["action_id", "xG"])\
                                                    .orderBy(col("xG").desc())\
        #xG_df tìm trong k điểm đó thì có bao nhiêu lần bàn thắng được ghi
        return xG_df
    
    def RMSE(self):
        k = self.k
        w = self.train_vec.orderBy(f.rand())
        test_size = int(w.count()/4)
        test_df = w.limit(test_size)
        train_df = w.subtract(test_df)
        test_df = test_df.withColumnRenamed("feature", "test_feature")\
                                    .withColumn("test_id", monotonically_increasing_id())
        
        actual_goal_df = test_df.select(col("test_id"), col("is_goal").alias("actual_goal"))
        
        join_df = train_df.crossJoin(test_df)
    
        cos_map_df = join_df\
                    .rdd.map(lambda x: (float(cos(x.feature,x.test_feature)), x.is_goal, x.test_id))\
                    .toDF(["cosine_score", "is_goal", "test_id"])
    
        windowDept = Window.partitionBy("test_id").orderBy(col("cosine_score").desc(), col("is_goal").desc())
    
        top_k_nearest_items_df = cos_map_df\
                                .withColumn("row",row_number().over(windowDept)).filter(col("row") <= k)
    
        xG_df = top_k_nearest_items_df.rdd.map(lambda x: (x.test_id, x.cosine_score*x.is_goal))\
                                                    .reduceByKey(add)\
                                                    .map(lambda x: (x[0], x[1]/k))\
                                                    .toDF(["test_id", "xG"])
        
        join_df = xG_df.join(actual_goal_df, ["test_id"])
        #map df test và cột actual goal để xem dự đoán có chính xác không

        result_rdd = join_df.rdd.map(lambda x: ((x.xG - x.actual_goal)**2))
        
        return (result_rdd.reduce(add)/result_rdd.count())**(1/2)

In [57]:
a = KNN_goal_predict("DEU",train_vec,test_vec, 3)

In [51]:
a.RMSE()

0.5946251221929777

In [ ]:
a.Accuracy()

In [52]:
result=a.knn_result().show()

+---------+------------------+
|action_id|                xG|
+---------+------------------+
|      852|0.9990796033141606|
|      693|0.9988870110474668|
|      253|0.9987624273754525|
|      447|0.9985743478145125|
|      258|0.9983847952410857|
|      498|0.9981753214263681|
|       33|0.9979990327176892|
|      557|0.9979302879169607|
|       83|0.9977854642937957|
|      488|  0.99775347673057|
|      311|0.9967637060704321|
|      110|0.9966888375059261|
|       63|0.9962791742877334|
|      453|0.9960378615328613|
|      648| 0.995158381917434|
|      786|0.9947790090848381|
|      165|0.9946544813095984|
|      665|0.9943607115366012|
|      400|0.6661851660130832|
|      605|0.6656223213047716|
+---------+------------------+
only showing top 20 rows



In [60]:
action_df.filter("ActionID==33").select("ActionID","feature").toPandas()
#"shot_place","shot_outcome","location","bodypart","assist_method","situation","is_goal

,ActionID,feature
0,33,"[4, 1, 9, 2, 1, 3, 1]"


In [ ]:
#shot_place: 10 Too high
#shot_outcome: 2 Off target
#location:11 Right side of the box
#bodypart: rightfoot
#assis method: headed pass
